In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

In [2]:
import os
import sys
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

# tornar o pyspark "importável"
findspark.init()
findspark.find()

'/content/spark-3.5.3-bin-hadoop3'

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("quetion_two") \
    .getOrCreate()

dados_employee = [
    ('1', 'John', 'Doe', 10000, '2023-01-01', 'Engineering'),
    ('2', 'Jane', 'Smith', 12000, '2022-12-01', 'Marketing'),
    ('3', 'Alice', 'Johnson', 12000, '2022-11-01', 'Engineering'),
    ('4', 'Bob', 'Brown', 15000, '2021-05-01', 'Sales'),
    ('5', 'Charlie', 'Green', 13000, '2022-03-01', 'HR'),
    ('6', 'David', 'White', 16000, '2020-07-01', 'Engineering'),
    ('7', 'Eve', 'Black', 14000, '2023-06-01', 'Marketing'),
    ('8', 'Frank', 'Blue', 12000, '2022-08-01', 'Sales'),
    ('9', 'Grace', 'Pink', 11000, '2023-02-01', 'Marketing'),
    ('10', 'Helen', 'Yellow', 16000, '2021-12-01', 'Engineering')
]

dados_titles = [
    ('1', 'Engineer', '2022-01-01'),
    ('2', 'Manager', '2022-01-01'),
    ('3', 'Engineer', '2022-01-01'),
    ('4', 'Sales Lead', '2021-05-01'),
    ('5', 'HR Manager', '2022-03-01'),
    ('6', 'Senior Engineer', '2020-07-01'),
    ('7', 'Marketing Specialist', '2023-06-01'),
    ('8', 'Sales Associate', '2022-08-01'),
    ('9', 'Marketing Analyst', '2023-02-01'),
    ('10', 'Senior Engineer', '2021-12-01')
]


colunas_employee = ['worker_id', 'first_name', 'last_name', 'salary', 'joining_date', 'department']
colunas_titles = ['worker_ref_id', 'worker_title', 'affected_from']


df_employee = spark.createDataFrame(dados_employee, schema=colunas_employee)
df_titles = spark.createDataFrame(dados_titles, schema=colunas_titles)


print("--- DataFrame Employee Criado ---")
df_employee.show()

print("--- Schema do DataFrame Employee ---")
df_employee.printSchema()

print("--- DataFrame Titles Criado ---")
df_titles.show()

print("--- Schema do DataFrame Titles ---")
df_titles.printSchema()


--- DataFrame Employee Criado ---
+---------+----------+---------+------+------------+-----------+
|worker_id|first_name|last_name|salary|joining_date| department|
+---------+----------+---------+------+------------+-----------+
|        1|      John|      Doe| 10000|  2023-01-01|Engineering|
|        2|      Jane|    Smith| 12000|  2022-12-01|  Marketing|
|        3|     Alice|  Johnson| 12000|  2022-11-01|Engineering|
|        4|       Bob|    Brown| 15000|  2021-05-01|      Sales|
|        5|   Charlie|    Green| 13000|  2022-03-01|         HR|
|        6|     David|    White| 16000|  2020-07-01|Engineering|
|        7|       Eve|    Black| 14000|  2023-06-01|  Marketing|
|        8|     Frank|     Blue| 12000|  2022-08-01|      Sales|
|        9|     Grace|     Pink| 11000|  2023-02-01|  Marketing|
|       10|     Helen|   Yellow| 16000|  2021-12-01|Engineering|
+---------+----------+---------+------+------------+-----------+

--- Schema do DataFrame Employee ---
root
 |-- worker_i

In [5]:
df_employee.createOrReplaceTempView("employee")
df_titles.createOrReplaceTempView("titles")

In [40]:
spark.sql("""
    SELECT worker_id, first_name, last_name, best_paid_title, salary
    FROM (
        SELECT e.worker_id, e.first_name, e.last_name, t.worker_title AS best_paid_title, e.salary,
               RANK() OVER (ORDER BY t.worker_title DESC) AS salary_rank
        FROM employee e
        JOIN titles t ON e.worker_id = t.worker_ref_id
    ) ranked_employees
    WHERE salary_rank = 1
""").show()

+---------+----------+---------+---------------+------+
|worker_id|first_name|last_name|best_paid_title|salary|
+---------+----------+---------+---------------+------+
|       10|     Helen|   Yellow|Senior Engineer| 16000|
|        6|     David|    White|Senior Engineer| 16000|
+---------+----------+---------+---------------+------+

